In [1]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

In [2]:
%reload_ext watermark
%watermark -a "Armindo Guerra"

Author: Armindo Guerra



In [5]:
import weaviate

client = weaviate.connect_to_local()

In [2]:
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

In [3]:
client.collections.delete("docs")

In [6]:
import weaviate.classes as wvc
from weaviate.classes.config import Property, DataType

# Create a new data collection
collection = client.collections.create(
    name = "docs", # Name of the data collection
    properties=[
        Property(name="text", data_type=DataType.TEXT), # Name and data type of the property
    ],
)

In [7]:
import ollama

# Store each document in a vector embedding database
with collection.batch.fixed_size(batch_size=200) as batch:
  for i, d in enumerate(documents):
    # Generate embeddings
    response = ollama.embeddings(model = "all-minilm",
                                 prompt = d)

    # Add data object with text and embedding
    batch.add_object(
        properties = {"text" : d},
        vector = response["embedding"],
    )

In [6]:
# An example prompt
prompt = "What animals are llamas related to?"

# Generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  model = "all-minilm",
  prompt = prompt,
)

results = collection.query.near_vector(near_vector = response["embedding"],
                                       limit = 1)

data = results.objects[0].properties['text']

In [7]:
prompt_template = f"Using this data: {data}. Respond to this prompt: {prompt}"

# Generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model = "llama2",
  prompt = prompt_template,
)

print(output['response'])


Llamas are members of the camelid family, which means they are closely related to two other animals: vicuñas and camels. These three species are all part of the Camelidae family and share many similarities in their physical characteristics and behavior. Vicuñas are wild relatives of llamas and alpacas, while camels are a different subfamily within the same family. So, to summarize, llamas are related to vicuñas and camels.


In [8]:
%watermark --iversions

weaviate: 4.11.1
ollama  : 0.4.7
IPython : 8.27.0

